# Converting sets of polygon coordinates to topojson files

Import packages

In [2]:
import pandas as pd
import ast
import copy

Load pandas table, first column is the id for the shape, second column is the coordinates transformed to geojson format using SQL query


##### Example of SQL Query  #####

select id, ST_AsGeoJSON(geom)
from
table_name ;;


In [18]:
query_results = pd.read_excel("/Users/leighajarett/Downloads/sql_runner_xbvrkqs2dysbs5_2019-07-24_21-10-48.xlsx")

Create the outer dictionary format, formatted so that the property key (field_id) can be used as a map layer in Looker

In [4]:
outer_dictionary = dict()
outer_dictionary["type"] = "Feature"
outer_dictionary["properties"] = dict()
outer_dictionary["properties"]["stroke"] = "#555555"
outer_dictionary["properties"]["stroke-width"] = 2
outer_dictionary["properties"]["stroke-opacity"] = 1
outer_dictionary["properties"]["fill"] = "#555555"
outer_dictionary["properties"]["fill-opacity"] = 0.5
outer_dictionary["properties"]["field_id"] = -1
outer_dictionary["geometry"] = None

In [5]:
outer_dictionary

{'type': 'Feature',
 'properties': {'stroke': '#555555',
  'stroke-width': 2,
  'stroke-opacity': 1,
  'fill': '#555555',
  'fill-opacity': 0.5,
  'field_id': -1},
 'geometry': None}

Make a list of the outer dictionary repeated and save it to the dataframe

In [6]:
outer_dictionary_list = [outer_dictionary] * len(query_results)
query_results['outer_dictionary'] = outer_dictionary_list

Convert the column to a dictionary

In [7]:
query_results['a'] = query_results['a'].apply(lambda x: ast.literal_eval(x))

Function to fill the outer dictionary values

In [8]:
def fill_dictionary(row):
    outer_dict = copy.deepcopy(row['outer_dictionary'])
    outer_dict['geometry'] = row['a']
    outer_dict['properties']['field_id'] = row['id']
    return outer_dict

In [9]:
query_results['filled_dictionary'] = query_results.apply(fill_dictionary, axis=1)
filled_dictionary_list = list(query_results['filled_dictionary'])

Create final dictionary and save as a geojson file

In [11]:
final_dictionary = dict()
final_dictionary['type'] = "FeatureCollection"
final_dictionary['features'] = filled_dictionary_list

In [12]:
geo_json_file = open("/Users/leighajarett/Desktop/tillable.geojson","w") 
geo_json_file.write(str(final_dictionary).replace("'",'"'))
geo_json_file.close()

Now you can convert this file to a topojson file using https://mapshaper.org/

Add this topojson as a map_layer in the lookml model file, specifying field_id as the property key, then this custom map_layer can be used with the dimension that corresponds to field_id